In [11]:
import imutils
import numpy as np
import cv2
import time

In [5]:
class Stitcher:
    def __init__(self):
        self.isv3=imutils.is_cv3()
    def stitch(self,images,ratio=0.75,reprojThresh=4.0,showMatches=False):
        # Unpack the images then detect the keypoints and extract local invariant descriptors from them
        (imageB,imageA)=images
        (kpsA,featuresA)=self.detectAndDescribe(imageA)
        (kpsB,featuresB)=self.detectAndDescribe(imageB)
        
        #match features between the two images
        M=self.matchKeypoints(kpsA,kpsB,featuresA,featuresB,ratio,reprojThresh)
        
        #if the match is None, then there aren't enough matched keypoints to create a panorama
        if M is None:
            return None
        
        # otherwise, apply a perspective warp to stitch the images together
        (matches,H,status)=M
        result=cv2.warpPerspective(imageA,H,(imageA.shape[1]+imageB.shape[1],imageA.shape[0]))
        result[0:imageB.shape[0],0:imageB.shape[1]]=imageB
        
        #check to see if the keypoint matches should be visualize
        if showMatches:
            vis=self.drawMatches(imageA,imageB,kpsA,kpsB,matches,status)
            # return a tuple of the stitched image and the visualization
            return (result,vis)
        #return the stitched images
        return result
    
    def detectAndDescribe(self,image):
        # convert the image to grayscale
        gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        #check to see if we are using opencv 3.x
        if self.isv3:
            #detect and extract the features from image
            descriptor=cv2.xfeatures2d.SIFT_create()
            (kps,features)=descriptor.detectAndCompute(image,None)
        # otherwise, we are using Opencv2.4.X
        else:
            #detect the keypoints in the image
            detector=cv2.FeatureDetector_create('SIFT')
            kps=detector.detect(gray)
            
            #extract features from keypoints in the image
            extractor=cv2.DescriptorExtractor_create('SIFT')
            (kps,features)=extractor.compute(gray,kps)
            
        #convert the keypoints from the keypoint objects to numpy arrays
        kps=np.float32([kp.pt for kp in kps])
        # return a tuple of keypoints and features
        return (kps,features)
    
    def matchKeypoints(self,kpsA,kpsB,featuresA,featuresB,ratio,reprojThresh):
        # compute the raw matches and initialize the list of actual matches
        matcher=cv2.DescriptorMatcher_create("BruteForce")
        rawMatches=matcher.knnMatch(featuresA,featuresB,2)
        matches=[]
        
        #loop over the raw matches
        for m in rawMatches:
            # ensure the distance is within a certain ratio of each other(i.e. Lowe's ratio test)
            if len(m)==2 and m[0].distance<m[1].distance*ratio:
                matches.append((m[0].trainIdx,m[1].queryIdx))
        # computing a homography requires at least 4 matches
        if len(matches)>4:
            #construct the two sets of points
            ptsA=np.float32([kpsA[i] for(_,i) in matches])
            ptsB=np.float32([kpsB[i] for(i,_) in matches])
            
            #computes the homography between the two sets of points
            (H,status)=cv2.findHomography(ptsA,ptsB,cv2.RANSAC,reprojThresh)
            
            #return the matches along with the homography matrix and status of each matched point
            return (matches,H,status)
        return None
    
    # visualize the keypoints corresponding between the two images
    def drawMatches(self,imageA,imageB,kpsA,kpsB,matches,status):
        #initialze the output visualization image
        (hA,wA)=imageA.shape[:2]
        (hB,wB)=imageB.shape[:2]
        vis=np.zeros((max(hA,hB),wA+wB,3),dtype='uint8')
        vis[0:hA,0:wA]=imageA
        vis[0:hB,wA:]=imageB
        
        # loop over all the matches
        for((trainIdx,queryIdx),s) in zip(matches,status):
            #only process the match if the keypoints was sucessfully matched
            if s==1:
                #draw the match
                ptA=(int(kpsA[queryIdx][0]),int(kpsA[queryIdx][1]))
                ptB=(int(kpsB[trainIdx][0])+wA,int(kpsB[trainIdx][1]))
                cv2.line(vis,ptA,ptB,(0,255,0),1)
        return vis

In [ ]:
imageA=cv2.imread('13.png')
imageB=cv2.imread('14.png')
imageA=imutils.resize(imageA,width=400)
imageB=imutils.resize(imageB,width=400)


stitcher=Stitcher()
start=time.clock()
(result,vis)=stitcher.stitch([imageA,imageB],showMatches=True)
eclipse=(time.clock()-start)

cv2.imshow('ImageA',imageA)
cv2.imshow('ImageB',imageB)
cv2.imshow('Keypoints',vis)
cv2.imshow('Result',result)
#cv2.imwrite('00_11_02.jpg',result)
cv2.waitKey(0)

In [35]:
cv2.destroyAllWindows()

In [3]:
cv2.xfeatures2d.SIFT_create()

<xfeatures2d_SIFT 000001E353F7A950>

In [14]:
eclipse

0.6779585215931885